In [ ]:
from pyspark.sql import SparkSession
from delta import *

def quiet_logs( sc ):
  logger = sc._jvm.org.apache.log4j
  logger.LogManager.getLogger("org"). setLevel( logger.Level.ERROR )
  logger.LogManager.getLogger("akka").setLevel( logger.Level.ERROR )

builder = (SparkSession.builder
          .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
          .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
          .appName('MyApp')
          .master('local[*]'))
spark = builder.getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions",1)

quiet_logs((spark.sparkContext))

KAFKA_BOOTSTRAP_SERVERS = "broker:29092"
KAFKA_TOPIC = "user-tracker"

from pyspark.sql.functions import col, concat, lit

kafka_df = spark.read.format("kafka").option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVERS)\
      .option('subscribe', KAFKA_TOPIC)\
      .option('startingOffsets', 'earliest')\
      .load()
    
kafka_df.select(col("offset"), col("value").cast("string")).show()